In [3]:
import os
import pandas as pd
import librosa
import soundfile as sf
import numpy as np
import math
import scipy.io.wavfile, scipy.signal
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
import plotly.express as px
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier



cols=['video_id', 'ts', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']

d=np.load('../resources/working_data/vocal_only_features.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv('../resources/dataset/lookup_new.csv')

vgg=pd.DataFrame(df["vggish"].to_list())
cols=[]
for col in vgg.columns:
    cols.append(f"vggish_{col+1}")
vgg.columns= cols

cols=['video_id', 'ts', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std']
for i in range(128):
    cols.append(f"vggish_{i+1}")


df=pd.concat([df, vgg], axis=1)
feature_df=df[cols]


from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='not minority',random_state=0)
from collections import Counter
X = feature_df[cols].to_numpy()
y=feature_df[['label']].to_numpy()

X_under, y_under = undersample.fit_resample(X, y)

undersampled_data = pd.DataFrame(X_under,columns=cols)
undersampled_data['label'] = y_under
undersampled_data

,video_id,ts,mid_ts,label,average_zcr,zcr_stddev,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,...,vggish_119,vggish_120,vggish_121,vggish_122,vggish_123,vggish_124,vggish_125,vggish_126,vggish_127,vggish_128
0,-2WqQY_xSSM,170.0,170.5,clean,0.079029,0.013053,-189.503776,206.294854,-97.415934,-1.505885,...,109.0,60.0,20.0,142.0,13.0,167.0,255.0,218.0,27.0,255.0
1,FNdC_3LR2AI,219.0,219.5,clean,0.031407,0.00892,-411.15477,189.961967,66.833366,19.346209,...,0.0,149.0,92.0,80.0,0.0,14.0,255.0,0.0,61.0,255.0
2,4600fGWcn9o,280.5,281.0,clean,0.043171,0.008926,-259.862115,117.961574,-48.594384,52.596798,...,112.0,119.0,64.0,211.0,61.0,58.0,230.0,0.0,45.0,255.0
3,get0cXOsSXg,80.0,80.5,clean,0.044832,0.00377,-339.360561,83.497913,-25.653152,-7.9201,...,0.0,232.0,238.0,148.0,117.0,215.0,207.0,28.0,215.0,255.0
4,74nTzbgDGWM,121.5,122.0,clean,0.117962,0.090615,-256.005722,136.614912,-104.303484,-11.822311,...,32.0,234.0,79.0,211.0,116.0,117.0,137.0,0.0,233.0,255.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,B7iIS91fMAc,117.0,117.5,no_vocals,0.06452,0.020594,-802.774922,78.86715,-23.590122,-25.774127,...,0.0,240.0,225.0,128.0,0.0,182.0,243.0,0.0,3.0,255.0
3326,ainbICPRV8Y,25.5,26.0,no_vocals,0.323871,0.035206,-942.420441,-21.156558,-19.957504,29.538438,...,47.0,219.0,103.0,121.0,103.0,81.0,168.0,145.0,68.0,255.0
3327,Bh_5ofa__pY,192.5,193.0,no_vocals,0.38137,0.034207,-832.679221,-85.070036,-27.147779,143.72803,...,23.0,51.0,107.0,105.0,57.0,131.0,180.0,66.0,0.0,255.0
3328,C_voh9WFbsM,193.0,193.5,no_vocals,0.37683,0.062925,-991.948038,-16.172109,2.334577,14.460796,...,47.0,216.0,102.0,116.0,103.0,88.0,174.0,151.0,71.0,255.0


In [3]:
cols

['video_id',
 'ts',
 'mid_ts',
 'label',
 'average_zcr',
 'zcr_stddev',
 'mfcc1_mean',
 'mfcc2_mean',
 'mfcc3_mean',
 'mfcc4_mean',
 'mfcc5_mean',
 'mfcc6_mean',
 'mfcc7_mean',
 'mfcc8_mean',
 'mfcc9_mean',
 'mfcc10_mean',
 'mfcc11_mean',
 'mfcc12_mean',
 'mfcc13_mean',
 'mfcc1_std',
 'mfcc2_std',
 'mfcc3_std',
 'mfcc4_std',
 'mfcc5_std',
 'mfcc6_std',
 'mfcc7_std',
 'mfcc8_std',
 'mfcc9_std',
 'mfcc10_std',
 'mfcc11_std',
 'mfcc12_std',
 'mfcc13_std',
 'delta_mfcc1_mean',
 'delta_mfcc2_mean',
 'delta_mfcc3_mean',
 'delta_mfcc4_mean',
 'delta_mfcc5_mean',
 'delta_mfcc6_mean',
 'delta_mfcc7_mean',
 'delta_mfcc8_mean',
 'delta_mfcc9_mean',
 'delta_mfcc10_mean',
 'delta_mfcc11_mean',
 'delta_mfcc12_mean',
 'delta_mfcc13_mean',
 'delta_mfcc1_std',
 'delta_mfcc2_std',
 'delta_mfcc3_std',
 'delta_mfcc4_std',
 'delta_mfcc5_std',
 'delta_mfcc6_std',
 'delta_mfcc7_std',
 'delta_mfcc8_std',
 'delta_mfcc9_std',
 'delta_mfcc10_std',
 'delta_mfcc11_std',
 'delta_mfcc12_std',
 'delta_mfcc13_std',
 '

In [8]:
import plotly.io as pio
pio.renderers.default = "vscode"

In [9]:
import plotly.express as px
lst=undersampled_data.columns
lst=[i for i in lst if i not in ['video_id', 'ts', 'mid_ts']]
fig = px.scatter_matrix(undersampled_data[lst],dimensions=["zcr_stddev", "rms_mean",'rms_std', "mfcc1_mean","flatness_mean","flatness_std"],color='label')
fig.show()

In [10]:
import plotly.express as px
lst=undersampled_data.columns
lst=[i for i in lst if i not in ['video_id', 'ts', 'mid_ts']]
fig = px.scatter_matrix(undersampled_data[lst],dimensions=['vggish_1',
 'vggish_10',
 'vggish_20',
 'vggish_30',
 'vggish_40','rms_mean','mfcc1_mean','flatness_mean','flatness_std'],color='label')
fig.show()

In [11]:
import plotly.express as px
lst=undersampled_data.columns
lst=[i for i in lst if i not in ['video_id', 'ts', 'mid_ts']]
fig = px.scatter_matrix(undersampled_data[lst],dimensions=['average_zcr',
 'zcr_stddev','centroid_mean',
 'centroid_std',
 'contrast_mean',
 'contrast_std',
 'flatness_mean',
 'flatness_std',
 'rolloff_mean',
 'rolloff_std',
 'rms_mean',
 'rms_std'],color='label')
fig.show()

In [7]:
import plotly.express as px
lst=undersampled_data.columns
lst=[i for i in lst if i not in ['video_id', 'ts', 'mid_ts']]
fig = px.scatter_matrix(undersampled_data[lst],dimensions=['average_zcr',
 'zcr_stddev',
 'contrast_mean',
 'contrast_std'],color='label')
fig.show()